# SCRIPT TO PERFORM QUALITY CONTROL ON WHOLE-GENOME SEQUENCING DATA

In order to run, there has to be several files in the project folder:
- GENCODE GTF: Run Scripts/WGS/01_get_gencode_annotation.sh. Obtain from: https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_46/gencode.v46.annotation.gtf.gz (Check for newer versions).

Once completed, a new Jupyter Notebook should be initialized so we can access this file. Or unmount and mount again the project (https://community.ukbiobank.ac.uk/hc/en-gb/community/posts/16019592366365-It-seems-that-the-recently-dx-uploaded-files-does-not-show-up-on-mnt-project-until-I-re-start-the-whole-Jupyter-Lab-VM)


- PVCF BLOCKS: Run Notebooks/WGS/DragenBlockProcessing.ipynb. Obtain from: https://biobank.ndph.ox.ac.uk/ukb/ukb/auxdata/dragen_pvcf_coordinates.zip 
It needs parsing, but in https://github.com/HauserGroup/gogoGPCR2/tree/main/data/misc it is already parsed.

- Samples to remove file: Run Notebooks/WGS/01_QC_Samples.ipynb

If you find problems, run it with Spark Version: 2.3.1  (At least 8 nodes)

#### Initialization 
##### Load packages


Import to current directory: src/project_permed

In [ ]:
import dxpy
import pyspark

import hail as hl
from pathlib import Path
from datetime import datetime
import pandas as pd
import re

from matrixtables import smart_split_multi_mt

In [2]:
# Constants
DATABASE = "matrix_tables"
REFERENCE_GENOME = "GRCh38"
PROJ_NAME = "OPRM1"

Path("/tmp").resolve().mkdir(parents=True, exist_ok=True)

LOG_FILE = (
    Path("../hail_logs", f"{PROJ_NAME}_{datetime.now().strftime('%H%M')}.log")
    .resolve()
    .__str__()
)

#### Hail and spark configuration

In [3]:
# Spark init
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

# Create database in DNAX
spark.sql(f"CREATE DATABASE IF NOT EXISTS {DATABASE} LOCATION 'dnax://'")
mt_database = dxpy.find_one_data_object(name=DATABASE, classname="database")["id"]

# Hail init
hl.init(sc=sc, default_reference=REFERENCE_GENOME, log=LOG_FILE)

pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/backend/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/backend/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 3.2.3
SparkUI available at http://ip-10-60-53-200.eu-west-2.compute.internal:8081
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.116-cd64e0876c94
LOGGING: writing to /opt/hail_logs/OPRM1_0849.log


#### Variables

In [ ]:
# RAP
VCF_VERSION = "v1"
FIELD_ID = 24310  # DRAGEN population level WGS variants, pVCF format 500k release

# Paths
BULK_DIR = Path("/mnt/project/Bulk")

# Genes
GENES = ["OPRM1"]

### Quality control

#### Gene intervals and blocks 

In [5]:
# Get gene intervals
gene_interval = hl.experimental.get_gene_intervals(
    gene_symbols=GENES,
    reference_genome="GRCh38",
    gtf_file="file:///mnt/project/WGS_Lucia/WGS_QC/gencode.v46.annotation.gtf",
)
gene_interval

2025-05-23 08:49:41.997 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (not specified)
  Loading field 'f1' as type str (not specified)
  Loading field 'f2' as type str (not specified)
  Loading field 'f3' as type int32 (user-supplied)
  Loading field 'f4' as type int32 (user-supplied)
  Loading field 'f5' as type float64 (user-supplied)
  Loading field 'f6' as type str (not specified)
  Loading field 'f7' as type int32 (user-supplied)
  Loading field 'f8' as type str (not specified)
2025-05-23 08:50:01.263 Hail: INFO: wrote table with 3467156 rows in 12 partitions to /tmp/MtIAZBJKJ3eIoxiJtpAKnW
2025-05-23 08:50:05.343 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-05-23 08:50:10.319 Hail: INFO: get_gene_intervals found 1 entries:
gene: OPRM1 (ENSG00000112038)


[Interval(start=Locus(contig=chr6, position=154010496, reference_genome=GRCh38), end=Locus(contig=chr6, position=154246867, reference_genome=GRCh38), includes_start=True, includes_end=True)]

In [ ]:
# Get DRAGEN pVCF blocks
blocks = hl.import_table(
    "file:///mnt/project/WGS_Lucia/WGS_QC/dragen_pvcf_blocks.tsv", no_header=False
)
blocks = blocks.annotate(
    Chromosome=blocks.Chromosome.replace("23", "X").replace("24", "Y")
)
blocks = blocks.annotate(region=hl.str("").join([hl.str("chr"), blocks.Chromosome]))
blocks = blocks.annotate(
    interval=hl.locus_interval(
        blocks.region,
        hl.int32(blocks.Starting_Position),
        hl.int32(blocks.Ending_Position),
        reference_genome="GRCh38",
    )
).key_by("interval")

2025-05-23 08:50:11.647 Hail: INFO: Reading table without type imputation
  Loading field 'Row_Number' as type str (not specified)
  Loading field 'Chromosome' as type str (not specified)
  Loading field 'Block' as type str (not specified)
  Loading field 'Starting_Position' as type str (not specified)
  Loading field 'Ending_Position' as type str (not specified)


In [7]:
# Get blocks for given genes
gb = blocks.filter(hl.any(lambda inter: blocks.interval.overlaps(inter), gene_interval))
gb.show()

,,,,,,
Row_Number,Chromosome,Block,Starting_Position,Ending_Position,region,interval
str,str,str,str,str,str,interval<locus<GRCh38>>
"""60766""","""6""","""7700""","""153991433""","""154011429""","""chr6""",[chr6:153991433-chr6:154011429)
"""60767""","""6""","""7701""","""154011430""","""154031420""","""chr6""",[chr6:154011430-chr6:154031420)
"""60768""","""6""","""7702""","""154031421""","""154051414""","""chr6""",[chr6:154031421-chr6:154051414)
"""60769""","""6""","""7703""","""154051415""","""154071408""","""chr6""",[chr6:154051415-chr6:154071408)
"""60770""","""6""","""7704""","""154071409""","""154091406""","""chr6""",[chr6:154071409-chr6:154091406)
"""60771""","""6""","""7705""","""154091407""","""154111396""","""chr6""",[chr6:154091407-chr6:154111396)
"""60772""","""6""","""7706""","""154111397""","""154131395""","""chr6""",[chr6:154111397-chr6:154131395)
"""60773""","""6""","""7707""","""154131396""","""154151386""","""chr6""",[chr6:154131396-chr6:154151386)


#### Import vcf files of specific blocks

In [ ]:
VCF_DIR = Path(
    "DRAGEN WGS/DRAGEN population level WGS variants, pVCF format 500k release"
)

vcf_files = [
    f"file://{BULK_DIR / VCF_DIR}/{chromosome}/ukb{FIELD_ID}_c{chromosome.replace('chr', '')}_b{block}_{VCF_VERSION}.vcf.gz"
    for block, chromosome in zip(gb.Block.collect(), gb.region.collect())
]

mt = hl.import_vcf(
    vcf_files,
    drop_samples=False,
    reference_genome="GRCh38",
    array_elements_required=False,
    force_bgz=True,
)

2025-05-23 08:50:15.729 Hail: INFO: Coerced sorted dataset
2025-05-23 08:50:18.119 Hail: INFO: Coerced sorted dataset


In [9]:
# Only genes of interest
mt = hl.filter_intervals(mt, gene_interval)
print(f"{mt.count_rows()} variants after gene filtering")

2025-05-23 08:50:43.666 Hail: INFO: scanning VCF for sortedness...
2025-05-23 08:53:19.552 Hail: INFO: Coerced sorted VCF - no additional import work to do


78512 variants after gene filtering


In [10]:
# Remove singletons (variants that appear only once across all samples)
mt = mt.filter_rows(hl.agg.sum(mt.GT.n_alt_alleles()) > 1)
print(f"{mt.count_rows()} variants after removing singletons")

46265 variants after removing singletons


In [11]:
# First checkpoint
stage = "FIRST"
checkpoint_file = f"/tmp/{PROJ_NAME}.{stage}.cp.mt"

mt = mt.checkpoint(checkpoint_file, overwrite=True)

2025-05-23 09:07:43.587 Hail: INFO: wrote matrix table with 46265 rows and 490541 columns in 678 partitions to /tmp/OPRM1.FIRST.cp.mt


#### Multi-allele filtering

In [12]:
# Remove variants with 6 or more alleles
mt = mt.filter_rows(mt.alleles.length() <= 6)
print(f"{mt.count_rows()} variants with not more than 6 alleles")

46054 variants with not more than 6 alleles


In [13]:
# Split multi-allele variants into single ones
mt = smart_split_multi_mt(mt)
print(f"{mt.count_rows()} variants after multi-allele splitting")

59311 variants after multi-allele splitting


In [14]:
# Second checkpoint
stage = "SECOND"
checkpoint_file = f"/tmp/{PROJ_NAME}.{stage}.cp.mt"

mt = mt.checkpoint(checkpoint_file, overwrite=True)

2025-05-23 09:20:00.113 Hail: INFO: wrote matrix table with 59311 rows and 490541 columns in 1356 partitions to /tmp/OPRM1.SECOND.cp.mt


#### Quality control filtering

In [15]:
mt = mt.filter_entries(mt.FT == "PASS")

# Then, filter variants where there is at least one non-missing genotype
mt = mt.filter_rows(hl.agg.any(hl.is_defined(mt.GT)))

In [16]:
# third checkpoint
stage = "THIRD"
checkpoint_file = f"/tmp/{PROJ_NAME}.{stage}.cp.mt"

mt = mt.checkpoint(checkpoint_file, overwrite=True)

2025-05-23 09:22:13.277 Hail: INFO: wrote matrix table with 59311 rows and 490541 columns in 1356 partitions to /tmp/OPRM1.THIRD.cp.mt


In [ ]:
# Compute statistics about the number and fraction of filtered entries.
mt = hl.MatrixTable.compute_entry_filter_stats(
    mt, row_field="entry_stats_row", col_field="entry_stats_col"
)

In [18]:
# forth checkpoint
stage = "FORTH"
checkpoint_file = f"/tmp/{PROJ_NAME}.{stage}.cp.mt"

mt = mt.checkpoint(checkpoint_file, overwrite=True)

2025-05-23 09:29:40.545 Hail: INFO: wrote matrix table with 59311 rows and 490541 columns in 1356 partitions to /tmp/OPRM1.FORTH.cp.mt


In [ ]:
row_fraction_threshold = 0.95

# Filter variants where at least 95% of genotypes are unfiltered
mt = mt.filter_rows((1 - mt.entry_stats_row.fraction_filtered) > row_fraction_threshold)

In [ ]:
col_fraction_threshold = 0.95

# Filter samples where at least 95% of variants are unfiltered
mt = mt.filter_cols((1 - mt.entry_stats_col.fraction_filtered) > col_fraction_threshold)

In [21]:
# Five checkpoint
stage = "FIVE"
checkpoint_file = f"/tmp/{PROJ_NAME}.{stage}.cp.mt"

mt = mt.checkpoint(checkpoint_file, overwrite=True)

2025-05-23 09:33:34.175 Hail: INFO: wrote matrix table with 58863 rows and 490541 columns in 1356 partitions to /tmp/OPRM1.FIVE.cp.mt


#### Remove samples from 01_QC_Samples.ipynb

In [ ]:
samples_to_remove = hl.import_table(
    "file:///mnt/project/WGS_Lucia/Data/samples_to_remove.tsv", key="eid"
)

mt = mt.anti_join_cols(samples_to_remove)

# Filter rows (variants) where any sample information is still present
mt = mt.filter_rows(hl.agg.any(mt.GT.n_alt_alleles() > 0))

2025-05-23 09:33:36.300 Hail: INFO: Reading table without type imputation
  Loading field 'eid' as type str (not specified)


In [23]:
# Six checkpoint
stage = "SIX"
checkpoint_file = f"/tmp/{PROJ_NAME}.{stage}.cp.mt"

mt = mt.checkpoint(checkpoint_file, overwrite=True)

2025-05-23 09:38:28.278 Hail: INFO: wrote matrix table with 52078 rows and 431455 columns in 1356 partitions to /tmp/OPRM1.SIX.cp.mt


#### Variant Effect Predictor (VEP)

In [24]:
VEP_JSON = Path("GRCh38_VEP.json").resolve()

In [ ]:
mt = hl.vep(mt, f"file:{VEP_JSON}", block_size=100)

2025-05-23 09:41:09.982 Hail: INFO: wrote table with 52078 rows in 1356 partitions to /tmp/persist_TablelVfWNjw02e


In [ ]:
is_MANE = mt.aggregate_rows(
    hl.agg.all(hl.is_defined(mt.vep.transcript_consequences.mane_select))
)
assert is_MANE, "Selected transcript may not be MANE Select. Check manually."

mt = mt.annotate_rows(
    protCons=mt.vep.transcript_consequences.amino_acids[0].split("/")[0]
    + hl.str(mt.vep.transcript_consequences.protein_end[0])
    + mt.vep.transcript_consequences.amino_acids[0].split("/")[-1],
    varid=hl.variant_str(mt.locus, mt.alleles),
)

In [27]:
# Seven checkpoint
stage = "SEVEN"
checkpoint_file = f"/tmp/{PROJ_NAME}.{stage}.cp.mt"

mt = mt.checkpoint(checkpoint_file, overwrite=True)

2025-05-23 09:47:05.110 Hail: INFO: wrote matrix table with 52078 rows and 431455 columns in 1356 partitions to /tmp/OPRM1.SEVEN.cp.mt


### Filtering

In [28]:
mt = hl.variant_qc(mt)
mt = mt.filter_rows(mt.variant_qc.n_non_ref > 0)
mt = mt.filter_rows(mt.variant_qc.gq_stats.mean >= 20)
mt = mt.filter_rows(mt.variant_qc.call_rate >= 0.95)
mt = mt.filter_rows(mt.vep.most_severe_consequence != "intron_variant")
mt = mt.filter_rows(mt.vep.most_severe_consequence != "downstream_gene_variant")
mt = mt.filter_rows(mt.vep.most_severe_consequence != "upstream_gene_variant")

In [29]:
GENE = "OPRM1"

# gene=mt.vep.transcript_consequences.gene_symbol[0]
mt = mt.filter_rows(mt.vep.transcript_consequences.gene_symbol[0] == GENE)

In [30]:
print(f"{mt.count_rows()} variants after quality filtering")

3401 variants after quality filtering


In [31]:
# Eight checkpoint
stage = "EIGHT"
checkpoint_file = f"/tmp/{PROJ_NAME}.{stage}.cp.mt"

mt = mt.checkpoint(checkpoint_file, overwrite=True)

2025-05-23 09:53:44.531 Hail: INFO: wrote matrix table with 3401 rows and 431455 columns in 1356 partitions to /tmp/OPRM1.EIGHT.cp.mt


### Formating

In [32]:
qt = mt.rows()
qt = qt.explode(qt.vep.transcript_consequences)

qt = qt.select(
    qt.varid,
    qt.protCons,
    qt.vep.most_severe_consequence,
    qt.vep.transcript_consequences.protein_end,
    qt.vep.transcript_consequences.protein_start,
    qt.vep.transcript_consequences.amino_acids,
    qt.vep.transcript_consequences.gene_id,
    qt.vep.transcript_consequences.transcript_id,
    **qt.variant_qc.flatten(),
)

qt = qt.annotate(AC=qt.AC[1], AF=qt.AF[1], homozygote_count=qt.homozygote_count[1])
qt = qt.key_by().drop("locus", "alleles")

qt.show(5)

,,,,,,,,,,,,,,,,,,,,,,,,
varid,protCons,most_severe_consequence,protein_end,protein_start,amino_acids,gene_id,transcript_id,gq_stats.mean,gq_stats.stdev,gq_stats.min,gq_stats.max,AC,AF,AN,homozygote_count,call_rate,n_called,n_not_called,n_filtered,n_het,n_non_ref,het_freq_hwe,p_value_hwe,p_value_excess_het
str,str,str,int32,int32,str,str,str,float64,float64,float64,float64,int32,float64,int32,int32,float64,int64,int64,int64,int64,int64,float64,float64,float64
"""chr6:154010515:G:C""",NA,"""5_prime_UTR_variant""",NA,NA,NA,"""ENSG00000112038""","""ENST00000434900""",5.75e+01,1.38e+01,1.30e+01,9.90e+01,1,1.16e-06,862910,0,1.00e+00,431455,0,0,1,1,2.32e-06,5.00e-01,5.00e-01
"""chr6:154010520:G:A""",NA,"""5_prime_UTR_variant""",NA,NA,NA,"""ENSG00000112038""","""ENST00000434900""",5.75e+01,1.38e+01,1.30e+01,9.90e+01,2,2.32e-06,862908,0,1.00e+00,431454,0,1,2,2,4.64e-06,5.00e-01,5.00e-01
"""chr6:154010521:G:A""",NA,"""5_prime_UTR_variant""",NA,NA,NA,"""ENSG00000112038""","""ENST00000434900""",5.75e+01,1.38e+01,1.30e+01,9.90e+01,3,3.48e-06,862908,0,1.00e+00,431454,0,1,3,3,6.95e-06,5.00e-01,5.00e-01
"""chr6:154010530:C:T""",NA,"""5_prime_UTR_variant""",NA,NA,NA,"""ENSG00000112038""","""ENST00000434900""",5.75e+01,1.38e+01,1.30e+01,9.90e+01,3,3.48e-06,862910,0,1.00e+00,431455,0,0,3,3,6.95e-06,5.00e-01,5.00e-01
"""chr6:154010543:G:T""",NA,"""5_prime_UTR_variant""",NA,NA,NA,"""ENSG00000112038""","""ENST00000434900""",5.75e+01,1.38e+01,1.30e+01,9.90e+01,8,9.27e-06,862910,0,1.00e+00,431455,0,0,8,8,1.85e-05,5.00e-01,5.00e-01


In [33]:
# Group by each distinct 'most_severe_consequence' and count the number of occurrences
consequence_counts = qt.aggregate(
    hl.agg.group_by(qt.most_severe_consequence, hl.agg.count())
)

print(consequence_counts)

{'3_prime_UTR_variant': 2862, '5_prime_UTR_variant': 221, 'frameshift_variant': 10, 'inframe_deletion': 1, 'inframe_insertion': 1, 'missense_variant': 196, 'splice_donor_5th_base_variant': 2, 'splice_donor_region_variant': 4, 'splice_donor_variant': 4, 'splice_polypyrimidine_tract_variant': 13, 'splice_region_variant': 2, 'stop_gained': 10, 'synonymous_variant': 75}


#### Export 

In [34]:
qt.export("/tmp/variant_qc.tsv")
!hadoop fs -getmerge /tmp/variant_qc.tsv ../variant_qc.tsv
!dx upload ../variant_qc.tsv --path /WGS_Lucia/WGS_QC/Output/

2025-03-11 12:19:55.369 Hail: INFO: merging 1357 files totalling 697.1K...
2025-03-11 12:19:55.655 Hail: INFO: while writing:
    /tmp/variant_qc.tsv
  merge time: 285.219ms


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/cluster/hadoop/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/cluster/dnax/jars/dnanexus-api-0.1.0-SNAPSHOT-jar-with-dependencies.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Reload4jLoggerFactory]
2025-03-11 12:20:29,354 WARN metrics.MetricsReporter: Unable to initialize metrics scraping configurations from hive-site.xml. Message:InputStream cannot be null
2025-03-11 12:20:29,494 WARN service.DNAxApiSvc: Using default configurations. Unable to find dnanexus.conf.location=null
2025-03-11 12:20:29,495 INFO service.DNAxApiSvc: apiserver connection-pool config. MaxPoolSize=10, MaxPoolPerRoute=10,MaxWaitTimeout=60000
2025-03-11 12:20:29,495 INFO service.DNAxApiSvc: initializing http connection man

In [35]:
# BGEN file
BGEN_FILE = "/tmp/OPRM1"
GPs = hl.literal([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]])

mt = mt.annotate_entries(GP=GPs[mt.GT.n_alt_alleles()])

hl.export_bgen(
    mt=mt, varid=mt.varid, rsid=mt.varid, gp=mt.GP, output="file:" + BGEN_FILE
)

In [ ]:
# ANNOTATIONS file
ANNOTATIONS_FILE = "/tmp/OPRM1.annotations"

# List of variants of interest
variants_of_interest = [
    "G255E",
    "C192F",
    "M205T",
    "N152D",
    "S147C",
    "M74T",
    "R181C",
    "A104D",
    "N190K",
    "T155I",
]

annotations = (
    mt.select_rows(
        varid=mt.varid,
        gene=mt.vep.transcript_consequences.gene_symbol[0],
        annotation=hl.if_else(
            # Check if 'protCons' is in variants_of_interest
            hl.literal(variants_of_interest).contains(mt.protCons),
            mt.protCons,
            mt.vep.most_severe_consequence,
        ),
    )
    .rows()
    .key_by("varid")
    .drop("locus")
    .drop("alleles")
)

annotations.export("file:" + ANNOTATIONS_FILE, header=False)

2025-05-23 10:48:34.317 Hail: INFO: Coerced sorted dataset
2025-05-23 10:48:37.199 Hail: INFO: merging 55 files totalling 149.7K...
2025-05-23 10:48:37.282 Hail: INFO: while writing:
    file:/tmp/OPRM1.annotations
  merge time: 83.516ms


In [37]:
# SETLIST file
SETLIST_FILE = "/tmp/OPRM1.setlist"

position = mt.aggregate_rows(hl.agg.min(mt.locus.position))
names = mt.varid.collect()
names_str = ",".join(names)

gene_symbol = mt.vep.transcript_consequences.gene_symbol.collect()[0]
contig = mt.locus.contig.collect()[0]

# Ensure gene symbol is a string and not a list
if isinstance(gene_symbol, list):
    gene_symbol = gene_symbol[0]

line = f"{gene_symbol}\t{contig}\t{position}\t{names_str}"

with open(SETLIST_FILE, "w") as f:
    f.write(line)

In [38]:
bgen_file = BGEN_FILE + ".bgen"
sample_file = BGEN_FILE + ".sample"

!dx upload $bgen_file $sample_file $ANNOTATIONS_FILE $SETLIST_FILE --path /WGS_Lucia/WGS_QC/Output/

[===========================================================>] Uploaded 15,595,366 of 15,595,366 bytes (100%) /tmp/OPRM1.bgen
ID                                file-Gz82bFQJb4J610gbG3127990
Class                             file
Project                           project-GfVK998Jb4JJgVBjKXPyxJ9q
Folder                            /WGS_Lucia/WGS_QC/Output
Name                              OPRM1.bgen
State                             closing
Visibility                        visible
Types                             -
Properties                        -
Tags                              -
Outgoing links                    -
Created                           Tue Mar 11 12:21:02 2025
Created by                        luciass6
 via the job                      job-Gz813p8Jb4J97Fxpp15zKp2Y
Last modified                     Tue Mar 11 12:21:03 2025
Media type                        
archivalState                     "live"
cloudAccount                      "cloudaccount-dnanexus"
[=============

Upload AlphaMissense predictions for OPRM1. Obtained from https://alphamissense.hegelab.org/

In [34]:
!dx download "500k WGS:/WGS_Lucia/WGS_QC/Output/AlphaMissense.tsv"

[===========================================================>] Completed 657,418 of 657,418 bytes (100%) /opt/notebooks/AlphaMissense.tsvv


In [53]:
alphamissense_df = pd.read_csv("AlphaMissense.tsv", sep="\t")

In [54]:
print(alphamissense_df)

     # Uniprot ACC  Entry name Gene name         Ensemble id protein variant  \
0           P35372  OPRM_HUMAN     OPRM1  ENST00000330432.12       p.Asp2Ala   
1           P35372  OPRM_HUMAN     OPRM1  ENST00000330432.12       p.Asp2Cys   
2           P35372  OPRM_HUMAN     OPRM1  ENST00000330432.12       p.Asp2Glu   
3           P35372  OPRM_HUMAN     OPRM1  ENST00000330432.12       p.Asp2Phe   
4           P35372  OPRM_HUMAN     OPRM1  ENST00000330432.12       p.Asp2Gly   
...            ...         ...       ...                 ...             ...   
7576        P35372  OPRM_HUMAN     OPRM1  ENST00000330432.12     p.Pro400Ser   
7577        P35372  OPRM_HUMAN     OPRM1  ENST00000330432.12     p.Pro400Thr   
7578        P35372  OPRM_HUMAN     OPRM1  ENST00000330432.12     p.Pro400Val   
7579        P35372  OPRM_HUMAN     OPRM1  ENST00000330432.12     p.Pro400Trp   
7580        P35372  OPRM_HUMAN     OPRM1  ENST00000330432.12     p.Pro400Tyr   

     is_snv a.a.1  position a.a.2  path

In [ ]:
# Normalize variant format to match 'protCons', e.g., "D2A"
alphamissense_df["missense_code"] = (
    alphamissense_df["a.a.1"]
    + alphamissense_df["position"].astype(str)
    + alphamissense_df["a.a.2"]
)

In [90]:
print(alphamissense_df)

     # Uniprot ACC  Entry name Gene name         Ensemble id protein variant  \
0           P35372  OPRM_HUMAN     OPRM1  ENST00000330432.12       p.Asp2Ala   
1           P35372  OPRM_HUMAN     OPRM1  ENST00000330432.12       p.Asp2Cys   
2           P35372  OPRM_HUMAN     OPRM1  ENST00000330432.12       p.Asp2Glu   
3           P35372  OPRM_HUMAN     OPRM1  ENST00000330432.12       p.Asp2Phe   
4           P35372  OPRM_HUMAN     OPRM1  ENST00000330432.12       p.Asp2Gly   
...            ...         ...       ...                 ...             ...   
7576        P35372  OPRM_HUMAN     OPRM1  ENST00000330432.12     p.Pro400Ser   
7577        P35372  OPRM_HUMAN     OPRM1  ENST00000330432.12     p.Pro400Thr   
7578        P35372  OPRM_HUMAN     OPRM1  ENST00000330432.12     p.Pro400Val   
7579        P35372  OPRM_HUMAN     OPRM1  ENST00000330432.12     p.Pro400Trp   
7580        P35372  OPRM_HUMAN     OPRM1  ENST00000330432.12     p.Pro400Tyr   

     is_snv a.a.1  position a.a.2  path

In [ ]:
# ANNOTATIONS file 2 for mask 2 including AlphaMissense and LOFTE predictions
ANNOTATIONS_2_FILE = "/tmp/OPRM1.annotations_2"

variants_of_interest = [
    "G255E",
    "C192F",
    "M205T",
    "N152D",
    "S147C",
    "M74T",
    "R181C",
    "A104D",
    "N190K",
    "T155I",
]


annotations_2 = (
    mt.select_rows(
        varid=mt.varid,
        gene=mt.vep.transcript_consequences.gene_symbol[0],
        # annotation_1: protCons if missense, else fallback to most severe consequence
        annotation_1=hl.case()
        .when(mt.vep.most_severe_consequence == "missense_variant", mt.protCons)
        .default(mt.vep.most_severe_consequence),
        # annotation_2: if LoF present, use "LoF"
        # else if protCons is in the variants of interest, use protCons
        # else fallback to most_severe_consequence
        annotation_2=hl.case()
        .when(
            hl.is_defined(
                hl.find(lambda x: x.lof == "HC", mt.vep.transcript_consequences)
            ),
            "LoF",
        )
        .when(hl.literal(variants_of_interest).contains(mt.protCons), mt.protCons)
        .default(mt.vep.most_severe_consequence),
    )
    .rows()
    .key_by("varid")
    .drop("locus", "alleles")
)

annotations_2.export("file:" + ANNOTATIONS_2_FILE, header=True)

2025-05-23 12:33:40.613 Hail: INFO: Coerced sorted dataset
2025-05-23 12:33:43.569 Hail: INFO: merging 56 files totalling 213.6K...
2025-05-23 12:33:43.647 Hail: INFO: while writing:
    file:/tmp/OPRM1.annotations_2
  merge time: 77.376ms


In [ ]:
mt.select_rows(
    lof_set=hl.set(
        hl.map(
            lambda x: x.lof,
            hl.filter(lambda x: hl.is_defined(x.lof), mt.vep.transcript_consequences),
        )
    )
).rows().show(10)

,,
locus,alleles,lof_set
locus<GRCh38>,array<str>,set<str>
chr6:154010515,"[""G"",""C""]",{}
chr6:154010520,"[""G"",""A""]",{}
chr6:154010521,"[""G"",""A""]",{}
chr6:154010530,"[""C"",""T""]",{}
chr6:154010543,"[""G"",""T""]",{}
chr6:154010545,"[""T"",""C""]",{}
chr6:154010546,"[""G"",""A""]",{}
chr6:154010551,"[""G"",""GCCCT""]",{}


In [152]:
# Load exported annotations
annotations_2_df = pd.read_csv("/tmp/OPRM1.annotations_2", sep="\t", header=0)
print(annotations_2_df)

                    varid   gene         annotation_1         annotation_2
0      chr6:154010515:G:C  OPRM1  5_prime_UTR_variant  5_prime_UTR_variant
1      chr6:154010520:G:A  OPRM1  5_prime_UTR_variant  5_prime_UTR_variant
2      chr6:154010521:G:A  OPRM1  5_prime_UTR_variant  5_prime_UTR_variant
3      chr6:154010530:C:T  OPRM1  5_prime_UTR_variant  5_prime_UTR_variant
4      chr6:154010543:G:T  OPRM1  5_prime_UTR_variant  5_prime_UTR_variant
...                   ...    ...                  ...                  ...
3396   chr6:154132310:G:A  OPRM1  3_prime_UTR_variant  3_prime_UTR_variant
3397   chr6:154132324:T:C  OPRM1  3_prime_UTR_variant  3_prime_UTR_variant
3398  chr6:154132336:A:AT  OPRM1  3_prime_UTR_variant  3_prime_UTR_variant
3399   chr6:154132339:A:C  OPRM1  3_prime_UTR_variant  3_prime_UTR_variant
3400   chr6:154132340:C:T  OPRM1  3_prime_UTR_variant  3_prime_UTR_variant

[3401 rows x 4 columns]


In [ ]:
# Merge AlphaMissense info
annotations_2_df = annotations_2_df.merge(
    alphamissense_df[["missense_code", "pathogenicity class"]],
    left_on="annotation_1",
    right_on="missense_code",
    how="left",
)

print(annotations_2_df)

                    varid   gene         annotation_1         annotation_2  \
0      chr6:154010515:G:C  OPRM1  5_prime_UTR_variant  5_prime_UTR_variant   
1      chr6:154010520:G:A  OPRM1  5_prime_UTR_variant  5_prime_UTR_variant   
2      chr6:154010521:G:A  OPRM1  5_prime_UTR_variant  5_prime_UTR_variant   
3      chr6:154010530:C:T  OPRM1  5_prime_UTR_variant  5_prime_UTR_variant   
4      chr6:154010543:G:T  OPRM1  5_prime_UTR_variant  5_prime_UTR_variant   
...                   ...    ...                  ...                  ...   
3396   chr6:154132310:G:A  OPRM1  3_prime_UTR_variant  3_prime_UTR_variant   
3397   chr6:154132324:T:C  OPRM1  3_prime_UTR_variant  3_prime_UTR_variant   
3398  chr6:154132336:A:AT  OPRM1  3_prime_UTR_variant  3_prime_UTR_variant   
3399   chr6:154132339:A:C  OPRM1  3_prime_UTR_variant  3_prime_UTR_variant   
3400   chr6:154132340:C:T  OPRM1  3_prime_UTR_variant  3_prime_UTR_variant   

     missense_code pathogenicity class  
0              NaN    

In [ ]:
# Use AlphaMissense value if available, else keep annotation_2
annotations_2_df["annotation"] = annotations_2_df.apply(
    lambda row: row["pathogenicity class"]
    if row["annotation_2"] == "missense_variant"
    else row["annotation_2"],
    axis=1,
)

# Keep only final output columns
annotations_2_df = annotations_2_df[["varid", "gene", "annotation"]]
print(annotations_2_df)

                    varid   gene           annotation
0      chr6:154010515:G:C  OPRM1  5_prime_UTR_variant
1      chr6:154010520:G:A  OPRM1  5_prime_UTR_variant
2      chr6:154010521:G:A  OPRM1  5_prime_UTR_variant
3      chr6:154010530:C:T  OPRM1  5_prime_UTR_variant
4      chr6:154010543:G:T  OPRM1  5_prime_UTR_variant
...                   ...    ...                  ...
3396   chr6:154132310:G:A  OPRM1  3_prime_UTR_variant
3397   chr6:154132324:T:C  OPRM1  3_prime_UTR_variant
3398  chr6:154132336:A:AT  OPRM1  3_prime_UTR_variant
3399   chr6:154132339:A:C  OPRM1  3_prime_UTR_variant
3400   chr6:154132340:C:T  OPRM1  3_prime_UTR_variant

[3401 rows x 3 columns]


In [147]:
annotations_2_df.to_csv("OPRM1.annotations_2", sep="\t", header=False, index=False)

In [148]:
!dx upload OPRM1.annotations_2 --path /WGS_Lucia/WGS_QC/Output/

[===========================================================>] Uploaded 152,997 of 152,997 bytes (100%) OPRM1.annotations_2
ID                                file-J0j6F60Jb4J31P044Xz4jbX1
Class                             file
Project                           project-GfVK998Jb4JJgVBjKXPyxJ9q
Folder                            /WGS_Lucia/WGS_QC/Output
Name                              OPRM1.annotations_2
State                             closing
Visibility                        visible
Types                             -
Properties                        -
Tags                              -
Outgoing links                    -
Created                           Fri May 23 12:10:00 2025
Created by                        luciass6
 via the job                      job-J0j2z5jJb4JKq7ZgzbXj1jJy
Last modified                     Fri May 23 12:10:01 2025
Media type                        
archivalState                     "live"
cloudAccount                      "cloudaccount-dnanexus"


## Obtain the eids per masks 

First we want to only keep the variants with annotations of interest to be faster. So we will take away UTR variants that have no predicted significant effect

In [ ]:
mt = mt.filter_rows(
    ~hl.literal({"5_prime_UTR_variant", "3_prime_UTR_variant"}).contains(
        mt.vep.most_severe_consequence
    )
)

In [39]:
print(f"{mt.count_rows()} variants after filtering")

318 variants after filtering


In [38]:
# Eight checkpoint
stage = "NINE"
checkpoint_file = f"/tmp/{PROJ_NAME}.{stage}.cp.mt"

mt = mt.checkpoint(checkpoint_file, overwrite=True)

2025-04-25 16:38:09.233 Hail: INFO: wrote matrix table with 318 rows and 431455 columns in 1356 partitions to /tmp/OPRM1.NINE.cp.mt


In [ ]:
# PLINK file
PLINK_FILE = "/tmp/OPRM1"


hl.export_plink(mt, varid=mt.varid, output="file:" + PLINK_FILE)

2025-04-25 16:38:22.037 Hail: INFO: merging 1357 files totalling 32.7M...
2025-04-25 16:38:22.624 Hail: INFO: while writing:
    file:/tmp/OPRM1.bed
  merge time: 586.182ms
2025-04-25 16:38:22.807 Hail: INFO: merging 1356 files totalling 13.1K...
2025-04-25 16:38:23.007 Hail: INFO: while writing:
    file:/tmp/OPRM1.bim
  merge time: 199.567ms


In [41]:
bed_file = PLINK_FILE + ".bed"
bim_file = PLINK_FILE + ".bim"
fam_file = PLINK_FILE + ".fam"

!dx upload $bed_file $bim_file $fam_file --path /WGS_Lucia/WGS_QC/Output/

[===========================================================>] Uploaded 34,300,755 of 34,300,755 bytes (100%) /tmp/OPRM1.bed
ID                                file-J05qb1QJb4J3VZ1fKvK8jqb7
Class                             file
Project                           project-GfVK998Jb4JJgVBjKXPyxJ9q
Folder                            /WGS_Lucia/WGS_QC/Output
Name                              OPRM1.bed
State                             closing
Visibility                        visible
Types                             -
Properties                        -
Tags                              -
Outgoing links                    -
Created                           Fri Apr 25 16:38:30 2025
Created by                        luciass6
 via the job                      job-J05p34jJb4J9KGYffJb6Q5vz
Last modified                     Fri Apr 25 16:38:31 2025
Media type                        
archivalState                     "live"
cloudAccount                      "cloudaccount-dnanexus"
[===============

Now we need to "variants_per_eid.sh" to obtain "variants_per_eid.raw" with the OPRM1 plink files we just got

In [1]:
!dx download "500k WGS:/WGS_Lucia/WGS_QC/Output/variants_per_eid.raw"
!dx download "500k WGS:/WGS_Lucia/WGS_QC/Output/OPRM1.annotations"
!dx download "500k WGS:/WGS_Lucia/WGS_QC/Output/OPRM1.annotations_2"

[===========================================================>] Completed 282,611,414 of 282,611,414 bytes (100%) /opt/notebooks/variants_per_eid.raww
[===========================================================>] Completed 153,250 of 153,250 bytes (100%) /opt/notebooks/OPRM1.annotationss
[===========================================================>] Completed 152,997 of 152,997 bytes (100%) /opt/notebooks/OPRM1.annotations_22


In [ ]:
# Load the PLINK output file (tab-separated) with the eids and variants of interest
file_path = "variants_per_eid.raw"  # Replace with actual file path

df = pd.read_csv(file_path, sep="\t", dtype={"IID": str})

# Extract the EID (IID) column and the variant columns (everything after the 6th column)
eid_column = "IID"
variant_columns = df.columns[
    6:
]  # Ignore first 6 columns (FID, IID, PAT, MAT, SEX, PHENOTYPE)

In [ ]:
# Function to clean variant names by removing the extra _A, _C, _G, _T
def clean_variant_name(variant):
    return re.sub(r"_[ACGT]+$", "", variant)  # Removes _A, _C, _G, _T


# Create a dictionary to store variants and their corresponding EIDs
variant_to_eids = {}

# Iterate over each row in the DataFrame
for _, row in df.iterrows():
    eid = row[eid_column]  # Get the EID
    for var in variant_columns:
        genotype = row[var]  # Get genotype value (0, 1, or 2)
        if genotype != 2:  # Only keep non-reference variants (1 or 2)
            clean_var = clean_variant_name(var)  # Clean the variant name
            if clean_var not in variant_to_eids:
                variant_to_eids[clean_var] = []  # Initialize list
            variant_to_eids[clean_var].append(eid)  # Add EID to the variant

In [ ]:
# Convert dictionary to DataFrame
variant_eid_df = pd.DataFrame(
    list(variant_to_eids.items()), columns=["Variant", "EIDs"]
)

# Ensure unique EIDs per variant
variant_eid_df["EIDs"] = variant_eid_df["EIDs"].apply(lambda x: list(set(x)))

In [ ]:
# Load the annotation file
annotation_df = pd.read_csv(
    "OPRM1.annotations_2",
    sep="\t",
    header=None,
    names=["Variant", "Gene", "Annotation"],
)

# Merge dataframes on the 'Variant' column, keeping only variants that are present in both dataframes
variant_eid_df = variant_eid_df.merge(
    annotation_df[["Variant", "Annotation"]], on="Variant", how="left"
)

# Reorder columns to place 'Annotation' as the second column
variant_eid_df = variant_eid_df[["Variant", "Annotation", "EIDs"]]

variant_eid_df["EIDs_Count"] = variant_eid_df["EIDs"].apply(len)

display(variant_eid_df)

,Variant,Annotation,EIDs,EIDs_Count
0,chr6:154039662:A:G,likely_benign,"[4712921, 3245475, 4641403, 4464719, 1002752, ...",103697
1,chr6:154039729:C:CCGG,inframe_insertion,"[5843227, 2125032, 5814061, 4439290, 4464719, ...",5505
2,chr6:154039685:G:A,synonymous_variant,"[5993865, 2757360, 1520292, 4771950, 4862223, ...",347
3,chr6:154089975:C:G,S147C,"[1283583, 4052941, 4434901, 4235822, 2692071, ...",7044
4,chr6:154090110:G:T,C192F,"[2601082, 4225123, 1336797, 4426077, 2959749, ...",6075
...,...,...,...,...
313,chr6:154091331:G:A,synonymous_variant,[2380332],1
314,chr6:154089987:A:G,likely_pathogenic,[5793424],1
315,chr6:154090942:T:C,splice_polypyrimidine_tract_variant,"[2022374, 2518086]",2
316,chr6:154090012:C:A,synonymous_variant,[5880941],1


In [ ]:
# Define your masks
# These are the defined for morphine efficacy
masks = {
    "No_efficacy": ["G255E", "R181C", "A104D", "N190K"],
    "WT_efficacy": ["S147C", "N152D", "T155I", "C192F", "M205T"],
    "No_efficacy_High_Impact_Variants": [
        "G255E",
        "R181C",
        "A104D",
        "N190K",
        "frameshift_variant",
        "stop_gained",
        "splice_donor_variant",
    ],
    "No_efficacy_High_Moderate_Impact_Variants": [
        "G255E",
        "R181C",
        "A104D",
        "N190K",
        "frameshift_variant",
        "stop_gained",
        "splice_donor_variant",
        "missense_variant",
        "inframe_deletion",
        "inframe_insertion",
    ],
}

masks_2 = {
    "No_efficacy_variants": ["G255E", "R181C", "A104D", "N190K"],
    "High_Impact_variants": [
        "frameshift_variant",
        "stop_gained",
        "splice_donor_variant",
    ],
    "Pathogenic_variants": ["likely_pathogenic"],
    "High_Impact_Pathogenic_variants": [
        "frameshift_variant",
        "stop_gained",
        "splice_donor_variant",
        "likely_pathogenic",
    ],
    "Benign_Ambiguous_variants": ["likely_benign", "ambiguous"],
    "Benign_Synonymous_variants": ["synonymous_variant", "likely_benign"],
    "Benign_Ambiguous_Synonymus_variants": [
        "likely_benign",
        "ambiguous",
        "synonymous_variant",
    ],
    "Synonymus_variants": ["synonymous_variant"],
    "WT_efficacy_variants": ["S147C", "N152D", "T155I", "C192F", "M205T"],
}

lovo_mask = {
    "No_efficacy_variants": ["G255E", "R181C", "A104D", "N190K"],
    "No_efficacy_variants_no_G255E": ["R181C", "A104D", "N190K"],
    "No_efficacy_variants_no_R181C": ["G255E", "A104D", "N190K"],
    "No_efficacy_variants_no_A104D": ["G255E", "R181C", "N190K"],
    "No_efficacy_variants_no_N190K": ["G255E", "R181C", "A104D"],
}

In [ ]:
# List to collect result rows
rows = []

# Loop through each mask and annotation list
for mask_name, annots in lovo_mask.items():
    # Filter rows where the annotation is in the mask
    matching = variant_eid_df[variant_eid_df["Annotation"].isin(annots)]

    # Flatten EIDs and add the mask name
    eids = []
    for _, row in matching.iterrows():
        eids.extend(row["EIDs"])

    # Keep unique EIDs
    unique_eids = list(set(eids))

    # Append the mask, its list of unique EIDs, and the count
    rows.append({"Mask": mask_name, "EIDs": unique_eids, "EID_Count": len(unique_eids)})

# Create final DataFrame
mask_eid_df = pd.DataFrame(rows)

# Preview
print(mask_eid_df)

                            Mask  \
0           No_efficacy_variants   
1  No_efficacy_variants_no_G255E   
2  No_efficacy_variants_no_R181C   
3  No_efficacy_variants_no_A104D   
4  No_efficacy_variants_no_N190K   

                                                EIDs  EID_Count  
0  [2519137, 4689076, 3756863, 1238379, 5558845, ...       1864  
1  [2519137, 4689076, 3756863, 1238379, 5558845, ...       1824  
2  [3447135, 4044114, 4617154, 4046254, 4727486, ...         69  
3  [2519137, 4689076, 3756863, 1238379, 5558845, ...       1847  
4  [2519137, 4689076, 3756863, 1238379, 5558845, ...       1852  


In [14]:
# Save DataFrame
mask_eid_df.to_csv("lovo_mask_morphine_eid_df.tsv", sep="\t", index=False)

In [15]:
# Upload DataFrame
!dx upload lovo_mask_morphine_eid_df.tsv --path WGS_Lucia/WGS_QC/Output/

[===========================================================>] Uploaded 82,205 of 82,205 bytes (100%) lovo_mask_morphine_eid_df.tsv
ID                                file-J0jqJp0Jb4J19GJg4pG64Px8
Class                             file
Project                           project-GfVK998Jb4JJgVBjKXPyxJ9q
Folder                            /WGS_Lucia/WGS_QC/Output
Name                              lovo_mask_morphine_eid_df.tsv
State                             closing
Visibility                        visible
Types                             -
Properties                        -
Tags                              -
Outgoing links                    -
Created                           Sat May 24 12:09:12 2025
Created by                        luciass6
 via the job                      job-J0jpqP0Jb4JKbGq5Jxyjx2qG
Last modified                     Sat May 24 12:09:13 2025
Media type                        
archivalState                     "live"
cloudAccount                      "cloudaccount